# PSR Factory Tutorial

### Install the most recent version from PyPI.org

In [ ]:
%pip install psr-factory

Requirements:
- Python 3.9 or greater
- Windows 10 or newer
    - Windows 7 SP1 up to Windows 8 with Visual C++ Redistributables 2015-22
- Linux, LIBC 2.28 or greater

Optional Python packages (DataFrames handling):
- numpy
- pandas
- polars


### User manual and examples

https://docs.psr-inc.com/factory/

### Downloadable versions (`pip`-less and 4.0 only currently)

https://psrenergy-docs.github.io/factory/releases.html

### Import the module to start using it

In [ ]:
import psr.factory
psr.factory.version()

### Create a case from scratch

The Factory script below will create, in memory, a case similar to Case21 found in SDDP 18 installation. 

In [ ]:
import psr.factory.samples.sddp_sample_case21 as sample_case

study = sample_case.create_sddp_sample_case21()

The code below will list all ThermalPlants attached to the study

In [ ]:
study.find("ThermalPlant")

The code below will list all Thermal plants with name starting with `T1`

In [ ]:
study.find("ThermalPlant.T1")

You can use a _glob_ pattern including wildcards such as `*` (one or more character) or `?` (one character)

In [ ]:
study.find("ThermalPlant.T*")

In [ ]:
# in this case, both will have de same result
study.find("ThermalPlant.T?")

Let's check T1

In [ ]:
t1 = study.find("ThermalPlant.T1")[0]
print(f"Code {t1.code}")
print(f"Name {t1.name}")
print(f"Type {t1.type}")

Let's traverse more plants in the case

In [ ]:
for t in study.find("ThermalPlant.*"):
    print(f"T {t.code}, {t.name}, {t.type}")

In [ ]:
for t in study.find("HydroPlant.*"):
    print(f"H {t.code}, {t.name}, {t.type}")

Lets go back to T1 and check its Installed Capacity

Properties that are not identifiers, are read using `get`, and set using... `set`

In [ ]:
t1.get("InstalledCapacity")

Improving the loop to show the initial installed capacity

In [ ]:
for t in study.find("ThermalPlant.*") + study.find("HydroPlant.*"):
    if t.type == "ThermalPlant":
        installed_capacity = t.get("InstalledCapacity")
        print(f"T {t.code}, {t.name}, {t.type}, {installed_capacity} MW")
    else:
        installed_capacity = t.get("InstalledCapacity")
        print(f"H {t.code}, {t.name}, {t.type}, {installed_capacity} MW")

Relationship between DataObject are set with special properties called `References`. A `ThermalPlant` is related to a `System` through the `RefSystem` relationship


```mermaid
flowchart LR
    ThermalPlant -->|RefSystem| System
```

So we will read it by using `t1.get('RefSystem')`

In [ ]:
system = t1.get('RefSystem')
system

Improving our example by showing the system the plant is part of

In [ ]:
for plant in study.find("ThermalPlant") + study.find("HydroPlant"):
    system = plant.get("RefSystem")  # <-- this is how you read the related system
    print(plant.type, plant.code, plant.name, system.name)

Listing all objects in the study

In [ ]:
for obj in study.get_all_objects():
    print(obj.type, obj.code, obj.name)

#### Exploring other properties

Use `properties` method to see whats is possible to read or set. Or go to the online user guide.

In [ ]:
count = 0
for prop_name, details in t1.properties().items():
    count += 1
    print(f"Property {prop_name}")
    print(f"  Type {details.type()}")
    print()
    if count > 5:
        break

`help()` shows similar information

In [ ]:
print(t1.help())

### Copying objects

In [ ]:
t4 = t1.clone()
t4.code = 4
t4.name = "T4"
t4.set("InstalledCapacity", 20)

study.add(t4)

In [ ]:
study.find("ThermalPlant")

In [ ]:
import os
os.makedirs("test", exist_ok=True)
study.save("test")

In [ ]:
battery = psr.factory.create("Battery", study.context)
battery.code = 1
battery.name = "BESS 1"
battery.set("MaximumStorage", 10)
battery.set("MaximumCapacity", 10)
system = study.find("System")[0]
battery.set("RefSystem", system)
study.add(battery)

In [ ]:
study.find("Battery")

In [ ]:
study.remove(t4)
study.find("ThermalPlant")

In [ ]:
battery.set_at("MaximumCapacity", "06/2025", 20)
battery.get_df("MaximumCapacity")

In [ ]:
demand_segment = study.find("DemandSegment")[0]


In [ ]:
demand_segment.get_df("EnergyPerBlock")

In [ ]:
demand_segment.set_df(demand_segment.get_df("EnergyPerBlock") * 2)